# Color Contrast Enhancement Demo
此 Notebook 示範如何手動將 RGB 影像轉換為 HSV，對 V 分量進行直方圖等化，再轉回 RGB，並透過 Gradio 建立互動式介面。

## 匯入所需的函式庫與定義核心函式
以下程式碼匯入必要的函式庫，並定義 RGB/HSV 轉換、V 分量等化與 Gradio 呼叫函式。

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import gradio as gr

def rgb_to_hsv_manual(img: np.ndarray):
    """
    手動將 RGB（0–255）轉到 HSV（h∈[0,1), s∈[0,1], v∈[0,1]）
    """
    # 1. 標準化到 [0,1]
    img_f = img.astype(np.float32) / 255.0
    r, g, b = img_f[...,0], img_f[...,1], img_f[...,2]
    c_max = np.maximum.reduce([r, g, b])
    c_min = np.minimum.reduce([r, g, b])
    delta = c_max - c_min

    # Hue 計算
    h = np.zeros_like(c_max)
    mask = delta > 1e-6
    # R 為 max
    idx = (c_max == r) & mask
    h[idx] = ((g[idx] - b[idx]) / delta[idx]) % 6
    # G 為 max
    idx = (c_max == g) & mask
    h[idx] = ((b[idx] - r[idx]) / delta[idx]) + 2
    # B 為 max
    idx = (c_max == b) & mask
    h[idx] = ((r[idx] - g[idx]) / delta[idx]) + 4
    h = h / 6  # 歸一化到 [0,1)

    # Saturation
    s = np.zeros_like(c_max)
    nonzero = c_max > 1e-6
    s[nonzero] = delta[nonzero] / c_max[nonzero]

    # Value
    v = c_max

    return np.stack([h, s, v], axis=-1)

def hsv_to_rgb_manual(hsv: np.ndarray):
    """
    手動將 HSV（h∈[0,1), s,v∈[0,1]）轉回 RGB（0–255）
    """
    h, s, v = hsv[...,0], hsv[...,1], hsv[...,2]
    h6 = h * 6
    i = np.floor(h6).astype(int) % 6
    f = h6 - i
    p = v * (1 - s)
    q = v * (1 - f * s)
    t = v * (1 - (1 - f) * s)

    rgb = np.zeros_like(hsv)
    for idx in range(6):
        mask = i == idx
        if idx == 0:
            rgb[mask] = np.stack([v[mask], t[mask], p[mask]], axis=-1)
        elif idx == 1:
            rgb[mask] = np.stack([q[mask], v[mask], p[mask]], axis=-1)
        elif idx == 2:
            rgb[mask] = np.stack([p[mask], v[mask], t[mask]], axis=-1)
        elif idx == 3:
            rgb[mask] = np.stack([p[mask], q[mask], v[mask]], axis=-1)
        elif idx == 4:
            rgb[mask] = np.stack([t[mask], p[mask], v[mask]], axis=-1)
        elif idx == 5:
            rgb[mask] = np.stack([v[mask], p[mask], q[mask]], axis=-1)

    # 轉回 [0,255]
    return (rgb * 255.0).clip(0,255).astype(np.uint8)

def enhance_contrast(img: np.ndarray):
    """整合流程：RGB→HSV→V 等化→RGB"""
    # 轉 HSV
    hsv = rgb_to_hsv_manual(img)
    # 對 v 通道做等化（先映射到 0–255，用 OpenCV）
    v_255 = (hsv[...,2]*255).astype(np.uint8)
    v_eq = cv2.equalizeHist(v_255)
    hsv[...,2] = v_eq.astype(np.float32)/255.0
    # 轉回 RGB
    enhanced = hsv_to_rgb_manual(hsv)
    return enhanced

def process(img: np.ndarray):
    """Gradio 呼叫函式"""
    enhanced = enhance_contrast(img)
    return img, enhanced

## 建立 Gradio 介面
以下程式碼建立 Gradio 介面，允許使用者上傳影像並即時顯示原始圖與增強後結果。

In [ ]:
iface = gr.Interface(
    fn=process,
    inputs=gr.Image(type="numpy", label="原始 RGB 影像"),
    outputs=[
        gr.Image(type="numpy", label="原始影像"),
        gr.Image(type="numpy", label="增強後影像")
    ],
    title="Homework 9：Color Contrast Enhancement",
    description="依照手動公式，將 RGB→HSV→對 V 分量做直方圖等化→再轉回 RGB。"
)

## 啟動介面
執行以下程式碼以啟動 Gradio 介面。

In [ ]:
if __name__ == "__main__":
    iface.launch()